In [4]:
import cv2 as cv

img = cv.imread('song.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray_x = cv.Sobel(gray, cv.CV_32F, 1,0,ksize=3)
gray_y = cv.Sobel(gray, cv.CV_32F, 0,1,ksize=3)

sobel_x = cv.convertScaleAbs(gray_x)
sobel_y = cv.convertScaleAbs(gray_y)

edge_strength = cv.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)

cv.imshow('Original', gray)
cv.imshow('sobelx', sobel_x)
cv.imshow('sobely', sobel_y)
cv.imshow('edge strength', edge_strength)

cv.waitKey()
cv.destroyAllWindows()

In [7]:
img = cv.imread('fast.jpg')
img = cv.resize(img, dsize=(0,0), fx=0.25, fy=0.25)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

canny1 = cv.Canny(gray,50,150)
canny2 = cv.Canny(gray,100,200)

cv.imshow('Original', gray)
cv.imshow('Canny1', canny1)
cv.imshow('Canny2', canny2)

cv.waitKey()
cv.destroyAllWindows()

In [8]:
import numpy as np
canny = cv.Canny(gray, 100, 200)
contour, hierarchy = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

lcontour = []
for i in range(len(contour)):
    if contour[i].shape[0] >100:
        lcontour.append(contour[i])

cv.drawContours(img, lcontour, -1, (0,255,0), 3)

cv.imshow('Original', img)
cv.imshow('Canny', canny)

cv.waitKey()
cv.destroyAllWindows()

In [11]:
img = cv.imread('room.jpg')
img = cv.resize(img, dsize=(0,0), fx=0.25, fy=0.25)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

apples = cv.HoughCircles(gray, cv.HOUGH_GRADIENT,1,200,param1=150, param2=20,minRadius=50, maxRadius=120)
for i in apples[0]:
    cv.circle(img, (int(i[0]), int(i[1])), int(i[2]), (255,0,0),2)
cv.imshow('Apple', img)
cv.waitKey()
cv.destroyAllWindows()

In [13]:
!pip install scikit-image

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.4 kB 326.8 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 401.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/24.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.5 MB 3.4 MB/s eta 0:00:08
   ---------------------------------------- 0.1/24.5 MB 1.1 MB/s eta 0:00:24
   ---------------------------------------- 0.2/24.5 MB 2.0 MB/s eta 0:00:13
    --------------------------------------- 0.4/24.5 MB 2.1 MB/s eta 0:00:12
    --------------------------------------- 0.5/24.5 MB 2.5 MB/s eta 0:00:10
   - -------------------------------------- 0.7/24.5 MB 2.7 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.5 MB 2.6 MB/s eta 0:00:10
   -- -------

In [24]:
import skimage
import numpy as np
import cv2 as cv

img = skimage.data.coffee()
cv.imshow('Coffee', cv.cvtColor(img, cv.COLOR_RGB2BGR))

slic1 = skimage.segmentation.slic(img, compactness=20, n_segments=600)
sp_img1 = skimage.segmentation.mark_boundaries(img, slic1)
sp_img1 = np.uint8(sp_img1 * 255.0)

slic2 = skimage.segmentation.slic(img, compactness=40, n_segments=600)
sp_img2 = skimage.segmentation.mark_boundaries(img, slic2)
sp_img2 = np.uint8(sp_img2 * 255.0)

cv.imshow('Super pixels (compact 20)', cv.cvtColor(sp_img1, cv.COLOR_RGB2BGR))
cv.imshow('Super pixels (compact 40)', cv.cvtColor(sp_img2, cv.COLOR_RGB2BGR))

cv.waitKey()
cv.destroyAllWindows()

In [25]:
!pip install -U scikit-image

In [26]:
import skimage
import numpy as np
import cv2 as cv
import time

coffee=skimage.data.coffee()  # coffee 영상을 읽어

start=time.time()  # 분할하는 데 걸리는 시간을 측정해 출력
slic=skimage.segmentation.slic(coffee,compactness=20,n_segments=600,start_label=1)
g=skimage.future.graph.rag_mean_color(coffee,slic,mode='similarity') 
ncut=skimage.future.graph.cut_normalized(slic,g)	# 정규화 절단
print(coffee.shape,' Coffee 영상을 분할하는데 ',time.time()-start,'초 소요')

marking=skimage.segmentation.mark_boundaries(coffee,ncut)  # 원래 영상인 coffee에 영역 분할 정보를 담은 ncut 맵을 이용하여 영역 경계를 표 시하고 marking 객체에 저장
ncut_coffee=np.uint8(marking*255.0)  # 0~1 사이의 실수를 가진 marking을 0~255 사이의 uint8 형으로 변환

cv.imshow('Normalized cut',cv.cvtColor(ncut_coffee,cv.COLOR_RGB2BGR))  # RGB로 표현된 분할 영상을 BGR로 변환하여 윈도우에 디스플레이

cv.waitKey()
cv.destroyAllWindows()

AttributeError: No skimage.future attribute graph